# Creating warm start forcing data

We want to add or replace soil data in the forcing data files. We show an example here, where we take the soil data from the output file of a remo run and add this to the forcing file of another run for use as initial lateral boundary conditions. 

In [49]:
import xarray as xr
from pyremo import remo_ds as rds
from pyremo import codes as rc

afile = xr.open_dataset("/work/ch0636/g300046/forcing-data/ERA5/EUR-11/xa/a056526a2002010100.nc")
tfile = rds.open_remo_dataset("/work/ch0636/g300046/forcing-data/ERA5/EUR-11/xa/e056524t2002010100", update_meta=True)

The output dataset (`tfile`) contains the spin up soil data. This could also come from another point in time of the same run or another run. The soil is defined by a number of codes or variable names:

In [52]:
soil_codes = [54,55,56,206,207,208,209,170,183,84,140,194,141]
soil_vars = [rc.get_dict(c)['variable'] for c in soil_codes]
soil_vars

['TSL',
 'TSW',
 'TSI',
 'TSN',
 'TD3',
 'TD4',
 'TD5',
 'TD',
 'TDCL',
 'QDBL',
 'WS',
 'WL',
 'SN']

In [ ]:
Now we copy and replace the data from the output file (`tfile`) in the forcing file (`afile`).

In [55]:
for var in soil_vars:
    afile[var] = tfile[var]

In [56]:
afile

<xarray.Dataset>
Dimensions:                     (lev: 27, nhyi: 28, rlat: 433, rlon: 433, time: 1)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
    lon                         (rlat, rlon) float64 ...
    lat                         (rlat, rlon) float64 ...
  * time                        (time) float64 2.002e+07
Dimensions without coordinates: lev, nhyi
Data variables: (12/23)
    rotated_latitude_longitude  int32 0
    hyai                        (nhyi) float64 0.0 5e+03 1e+04 ... 0.0 0.0 0.0
    hybi                        (nhyi) float64 0.0 0.0 0.0 ... 0.973 0.9923 1.0
    T                           (time, lev, rlat, rlon) float32 ...
    PS                          (time, rlat, rlon) float32 ...
    U                           (time, lev, rlat, rlon) float32 ...
    ...                          ...
    TD5                         (time, rlat, rlon) float32 ...
    TD                          (time, rlat, rlon) float32 ...
    TDCL                        (time, rlat, rlon) float32 ...
    WS                          (time, rlat, rlon) float32 ...
    WL                          (time, rlat, rlon) float32 ...
    SN                          (time, rlat, rlon) float32 ...
Attributes:
    driving_model_CDI:          Climate Data Interface version 1.9.7.1 (http:...
    driving_model_Conventions:  CF-1.6
    driving_model_history:      Fri Jun 04 14:56:03 2021: cdo -O -setname,ta ...
    driving_model_institution:  European Centre for Medium-Range Weather Fore...
    driving_model_CDO:          Climate Data Operators version 1.9.7.1 (http:...

In [57]:
afile.to_netcdf("/work/ch0636/g300046/forcing-data/ERA5/EUR-11/xa/a056526a2002010100_warm_soil.nc")